# The Elitzur-Vaidman bomb experiment

Your task is to improve the bomb-detecting strategy in this notebook to satisfy the following guarantee: given an error parameter $\varepsilon$, output an experiment that detects a bomb without triggering it with probability $1 - \varepsilon$.

In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from math import pi
from ibm_quantum_widgets import draw_circuit

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()
backend = Aer.get_backend('qasm_simulator')

In [2]:
b = QuantumCircuit(1, 1, name='Bomb')
b.measure(0, 0)
bomb = b.to_instruction()

nb = QuantumCircuit(1, 1, name='No bomb')
nb.id(0)
no_bomb = nb.to_instruction()

## Elitzur Vaidman solution with rotation gate

Considering $R_{\epsilon}$ such that, given state of qubit
$$
\ket{0} \implies R_{\theta} \ket{0} = \ket{\theta} = 
\cos (\theta) \ket{0} + \sin (\theta) \ket{1}
$$
Where 
$$
R_{\phi} \ket{theta} = \ket{\theta + \phi} = \cos (\theta + \phi) \ket{0} + \sin (\theta + \phi) \ket{1}
$$

The idea is to:
1. Send $\ket{0}$
2. Apply rotation gate with $\epsilon$ radians
3. send the qubit to the briefcase
4. Repeat steps 2 and 3 until the qubit is rotated to $\ket{1}$

This works since, if there is no bomb, no measurement is happening on the qubit and when we measure the resulting qubit, it is guaranteed to be measured as $\ket{1}$. Otherwise, everytime the bomb measures the qubit, the probability to explode is $O(\epsilon^2)$, which implies the resulting state will most likely be $\ket{0}$. This also implies, if we measure the qubit at the end, it will always be $\ket{0}$. This allows us to maximize our winning chances.

Since $\frac{\pi}{2}$ is irrational, $k * \epsilon \neq \frac{\pi}{2} \, \forall k \in \N$. To compensate for this, I ensure in the last iteration of the algorithm I add the remaining angle.

To do this I first argue the following lemma: 
$$
\begin{gather}
\forall a,b \in \R \text{ where : } b < a \\ 
b * \lceil \frac{a - b}{b}\rceil < a 
\end{gather}
$$
Proof:
Suppose $a / b = c$ where $c \in \N$. Then
$$
\begin{align}
b * \lceil \frac{a - b}{b}\rceil &=  b * \lceil \frac{a}{b} - 1 \rceil = b * (c - 1) < a
\end{align}
$$
Suppose $a / b = c$ where $c \notin \N$. Then $c = \lambda + f$ where $\lfloor a / b \rfloor = \lambda$ and $f = a - \lambda * b$. We know $f < 1$, therefore:
$$
\begin{align}
b * \lceil \frac{a - b}{b}\rceil &=  b * \lceil \lambda + f - 1 \rceil < b * \lambda < a
\end{align}
$$

Using the lemma I apply steps 2,3 with $R_{\epsilon}$, $\lceil \frac{\frac{\pi}{2} - \epsilon}{\epsilon}\rceil$ times and apply one last time $R_{\phi}$ where $\phi = \frac{\pi}{2} - \lceil \frac{\frac{\pi}{2} - \epsilon}{\epsilon}\rceil * \epsilon $

Using the following procedure, we can guarantee winning probability $O(\epsilon)$

In [1]:
def elitzur_vaidman(black_box, eps):
    # --------- #
    ### EDIT HERE ### 
    from math import ceil
    final = np.pi 
    iterations = int(ceil((final - eps) / eps)) # Based on the lemma explained in the markdown section
    circuit = QuantumCircuit(1,iterations + 2) # Add iterations +2 bits to measure the state of the bomb.
    # The reason for +2 is because, 1st bit will be used for the phi angle (explained in the previous section)
    # and the last bit will be used to measure the qubit itself

    # Reason for using multiple classical bits is: multiple bomb measures overwrite the current value on the register
    for i in range(iterations):
        circuit.ry(eps, 0) # apply epsilon rotation
        circuit.append(black_box, qargs=[0], cargs=[i]) # measure the bomb on the ith cbit
    circuit.ry(final - iterations * eps,0) # apply last iteration
    circuit.append(black_box, qargs=[0], cargs=[iterations])
    circuit.measure(0, iterations + 1) # This measures the bomb
    # --------- #
    
    
    job = backend.run(transpile(circuit, backend), shots=1024)
    results = {'Bomb': 0, 'BOOM!': 0, 'No bomb': 0}
    for outcome, frequency in job.result().get_counts().items():
        if '1' in outcome[1:]:
            results['BOOM!'] += frequency
        # --------- #
        ### EDIT HERE ###
        elif outcome[0] == '1': #Explained in the previous section. 
        # Essentially check: if the qubit was rotated to \ket{1} there is a bomb
        # else no bomb
        # --------- #     
            results['No bomb'] += frequency
        else:
            results['Bomb'] += frequency
    print(results)

In [4]:
for eps in [0.5, 0.1, 0.01, 0.001]:
    elitzur_vaidman(no_bomb, eps)

{'Bomb': 0, 'BOOM!': 0, 'No bomb': 1024}
{'Bomb': 0, 'BOOM!': 0, 'No bomb': 1024}
{'Bomb': 0, 'BOOM!': 0, 'No bomb': 1024}
{'Bomb': 0, 'BOOM!': 0, 'No bomb': 1024}


In [5]:
for eps in [0.5, 0.1, 0.01, 0.001]:
    elitzur_vaidman(bomb, eps)

{'Bomb': 693, 'BOOM!': 331, 'No bomb': 0}
{'Bomb': 940, 'BOOM!': 84, 'No bomb': 0}
{'Bomb': 1017, 'BOOM!': 7, 'No bomb': 0}
{'Bomb': 1023, 'BOOM!': 1, 'No bomb': 0}


In [9]:
# Feel free to include graphs/plots/visualisations here; but only the section in the elitzur_vaidman function will be marked.
from math import ceil
final = np.pi 
eps = 0.5
iterations = int(ceil((final - eps) / eps)) # Based on the lemma explain in the markdown section
circuit = QuantumCircuit(1,iterations + 2) # Add iterations +2 bits to measure the state of the bomb.
# The reason for +2 is because, +1th bit will be used for the phi angle (explained in the previous section)
# and the last bit will be used to measure the qubit itself
for i in range(iterations):
    circuit.ry(eps, 0) 
    circuit.append(no_bomb, qargs=[0], cargs=[i])
circuit.ry(final - iterations * eps,0)
circuit.append(no_bomb, qargs=[0], cargs=[iterations])
circuit.measure(0, iterations + 1) # This measures the bomb
circuit.draw()

┌─────────┐┌──────────┐┌─────────┐┌──────────┐┌─────────┐┌──────────┐»
  q: ┤ Ry(0.5) ├┤0         ├┤ Ry(0.5) ├┤0         ├┤ Ry(0.5) ├┤0         ├»
     └─────────┘│  No bomb │└─────────┘│          │└─────────┘│  No bomb │»
c_0: ═══════════╡0         ╞═══════════╡          ╞═══════════╡          ╞»
                └──────────┘           │          │           │          │»
c_1: ══════════════════════════════════╡0         ╞═══════════╡          ╞»
                                       └──────────┘           │          │»
c_2: ═════════════════════════════════════════════════════════╡0         ╞»
                                                              └──────────┘»
c_3: ═════════════════════════════════════════════════════════════════════»
                                                                          »
c_4: ═════════════════════════════════════════════════════════════════════»
                                                                          »
c_5: ═════════════════════════════════════════════════════════════════════»
                                                                          »
c_6: ═════════════════════════════════════════════════════════════════════»
                                                                          »
c_7: ═════════════════════════════════════════════════════════════════════»
                                                                          »
«     ┌─────────┐┌──────────┐┌─────────┐┌──────────┐┌─────────┐┌──────────┐»
«  q: ┤ Ry(0.5) ├┤0         ├┤ Ry(0.5) ├┤0         ├┤ Ry(0.5) ├┤0         ├»
«     └─────────┘│          │└─────────┘│          │└─────────┘│          │»
«c_0: ═══════════╡  No bomb ╞═══════════╡          ╞═══════════╡          ╞»
«                │          │           │  No bomb │           │          │»
«c_1: ═══════════╡          ╞═══════════╡          ╞═══════════╡  No bomb ╞»
«                │          │           │          │           │          │»
«c_2: ═══════════╡          ╞═══════════╡          ╞═══════════╡          ╞»
«                │          │           │          │           │          │»
«c_3: ═══════════╡0         ╞═══════════╡          ╞═══════════╡          ╞»
«                └──────────┘           │          │           │          │»
«c_4: ══════════════════════════════════╡0         ╞═══════════╡          ╞»
«                                       └──────────┘           │          │»
«c_5: ═════════════════════════════════════════════════════════╡0         ╞»
«                                                              └──────────┘»
«c_6: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«c_7: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«     ┌─────────────┐┌──────────┐┌─┐
«  q: ┤ Ry(0.14159) ├┤0         ├┤M├
«     └─────────────┘│          │└╥┘
«c_0: ═══════════════╡          ╞═╬═
«                    │          │ ║ 
«c_1: ═══════════════╡          ╞═╬═
«                    │  No bomb │ ║ 
«c_2: ═══════════════╡          ╞═╬═
«                    │          │ ║ 
«c_3: ═══════════════╡          ╞═╬═
«                    │          │ ║ 
«c_4: ═══════════════╡          ╞═╬═
«                    │          │ ║ 
«c_5: ═══════════════╡          ╞═╬═
«                    │          │ ║ 
«c_6: ═══════════════╡0         ╞═╬═
«                    └──────────┘ ║ 
«c_7: ════════════════════════════╩═
«